In [49]:
import pandas as pd
import numpy as np

In [50]:
matches = pd.read_csv("data/matchups.csv")

In [51]:
matches.head()

,P_MATCH_ID,GOLDEARNED,TOTALMINIONSKILLED,WIN,KILLS,ASSISTS,DEATHS,CHAMPION,VISIONSCORE,PUUID,TOTALDAMAGEDEALTTOCHAMPIONS,SUMMONERNAME,GAMEVERSION,Unnamed: 13
0,TR1_1201957752_top,14425,194,False,14,5,7,Kassadin,17,phduyQLB8gBjUerFwiVOtyLLHE9jxw7Jq7dwab_CtRddAv...,33426,Borke,11.14.384.6677,NaN
1,TR1_1201957752_top,12920,177,True,3,13,6,Fiora,17,SgQ5hm74vmXVDB3GIt3ka7gaBXFpNnT3PbHU8TbjEk5PJ8...,12092,Nora Durst,11.14.384.6677,NaN
2,TR1_1201957752_middle,13483,221,False,7,3,6,Yone,20,m7Sxq9HxRKLIgf0n0QOKhfHq8jNMZ-k1VhW_jcb307sUya...,20604,Fightêr Yone,11.14.384.6677,NaN
3,TR1_1201957752_middle,11375,167,True,4,15,7,Corki,27,pa-g2VGIEJn7CJCYYQU1PaHUqjS9pB7M4PsDcjM3ac5ysM...,30150,AM SORİİ HE,11.14.384.6677,NaN
4,TR1_1201957752_bottom,11320,197,False,5,10,12,Kaisa,17,_WeC06P4yIMW7ZMWi6bmwCagXp7c7bxxGv6nNW4--SGqXp...,16689,º J İ N X º,11.14.384.6677,NaN


## Data Description

     P_MATCH_ID - 
     GOLDEARNED
     TOTALMINIONSKILLED
     WIN
     KILLS
     ASSISTS
     DEATHS
     CHAMPION
     VISIONSCORE
     PUUID
     TOTALDAMAGEDEALTTOCHAMPIONS
     SUMMONERNAME
     GAMEVERSION


P_MATCH_ID contains 3 factors. Split them

In [52]:
matches[['Region', 'match_id', 'role']] = matches['P_MATCH_ID'].str.split('_', expand=True)

match_id is not unique among region, create an unique match_id

In [53]:
matches['match_id'] = matches['match_id'].str.cat(matches[['Region']], sep='_')

Drop unnecessary columns

In [54]:
matches = matches.drop(columns = ['P_MATCH_ID', 'Unnamed: 13', 'GAMEVERSION'])

In [55]:
matches.head()

,GOLDEARNED,TOTALMINIONSKILLED,WIN,KILLS,ASSISTS,DEATHS,CHAMPION,VISIONSCORE,PUUID,TOTALDAMAGEDEALTTOCHAMPIONS,SUMMONERNAME,Region,match_id,role
0,14425,194,False,14,5,7,Kassadin,17,phduyQLB8gBjUerFwiVOtyLLHE9jxw7Jq7dwab_CtRddAv...,33426,Borke,TR1,1201957752_TR1,top
1,12920,177,True,3,13,6,Fiora,17,SgQ5hm74vmXVDB3GIt3ka7gaBXFpNnT3PbHU8TbjEk5PJ8...,12092,Nora Durst,TR1,1201957752_TR1,top
2,13483,221,False,7,3,6,Yone,20,m7Sxq9HxRKLIgf0n0QOKhfHq8jNMZ-k1VhW_jcb307sUya...,20604,Fightêr Yone,TR1,1201957752_TR1,middle
3,11375,167,True,4,15,7,Corki,27,pa-g2VGIEJn7CJCYYQU1PaHUqjS9pB7M4PsDcjM3ac5ysM...,30150,AM SORİİ HE,TR1,1201957752_TR1,middle
4,11320,197,False,5,10,12,Kaisa,17,_WeC06P4yIMW7ZMWi6bmwCagXp7c7bxxGv6nNW4--SGqXp...,16689,º J İ N X º,TR1,1201957752_TR1,bottom


Check data for null values

In [56]:
matches[matches.isnull().any(axis=1)]

,GOLDEARNED,TOTALMINIONSKILLED,WIN,KILLS,ASSISTS,DEATHS,CHAMPION,VISIONSCORE,PUUID,TOTALDAMAGEDEALTTOCHAMPIONS,SUMMONERNAME,Region,match_id,role
398808,6051,37,True,0,10,0,Lulu,32,L2m7Wx4EYFqHCgGzHRz1IWpdjPY17m9epJyZI4_A1IswHo...,3502,NaN,NA1,3993107745_NA1,utility
1241011,17188,239,True,11,14,4,Kassadin,20,ZKqRsvGm94WBAZf8w_16AmPYZ33fHKKh440ARecW7Mf7C7...,34659,NaN,BR1,2186840449_BR1,middle
1303486,9579,11,True,4,18,1,Janna,56,L2m7Wx4EYFqHCgGzHRz1IWpdjPY17m9epJyZI4_A1IswHo...,11009,NaN,NA1,3990915292_NA1,utility


In [61]:
matches = matches[(matches.match_id != '3993107745_NA1') & (matches.match_id != '2186840449_BR1') & (matches.match_id != '3990915292_NA1')]

In [62]:
matches[matches.isnull().any(axis=1)]

,GOLDEARNED,TOTALMINIONSKILLED,WIN,KILLS,ASSISTS,DEATHS,CHAMPION,VISIONSCORE,PUUID,TOTALDAMAGEDEALTTOCHAMPIONS,SUMMONERNAME,Region,match_id,role
